# **Company Stock Market Sentimental Analysis**

**Hong Yee (Isaac) Hua**

---

Tired of sifting through analyst articles to gauge market sentiment and market news on a particular company?

**Look no further.**

This tool in Python is designed to obtain the most recent view of the market sentiment on a company, with a summarisation of recent market news.



**NOTE:** You will need a Cohere API key and an Alpha Vantage API key (obtainable through account creation). Maybe if you, dear reader, can get me a job, then you wouldn't need to provide your own API key.. however, I am but a lowly student with a trial key. Can't exactly process hundreds of articles.

## EXAMPLE INPUT AND OUTPUT:



> What is your Cohere API key?

>TqNim7S803C7Pz7pLmGYb38PZkiDeceA1wkFDDTf

>What is your AlphaVantage API key?

>KK6JVIIT1YZ51460

>What is your prospective company's ticker symbol?

>GOOG

>How many articles do you want to analyse? (Up to 200 for trial users)

>20

> Overall recent market sentiment:  

> Neutral 

> Summarization of current market news:  

> Alphabet Inc has added a new update to its music app that lets users automatically download recently played songs on Android. The new update allows users to download up to 200 recently played songs. The users may also find this feature already enabled on their devices. Meanwhile, Alphabet has announced that US creators can now create podcasts in YouTube Studio and the inclusion of podcasts in the company's music app is coming soon.

---
### **How does this work?**

Well, I'm glad you asked my friend, it's definitely been a journey to make this function.

Essentially,

1. Alpha Vantage AI pulls and processes analyst articles on the ticker, through websites such as Bezinga, Zacks etc.
2. Each article is processed through Newspaper to be read, and overall market sentiment is averaged out across all the articles through Alpha Vantage's natural language processing model.
3. Taking samples of each article, we summarise using Cohere AI's summarisation model. We feed these articles through layers and summarise further, depending on the complexity of articles.
4. Our end result is an average sentiment score across all the recent articles (number specified by user) and a final summarised paragraph of the most recent market news on the specified company.

Enjoy.

---

### **Improvements to be made:**
- Include foreign companies which are not in US stock market, use a translation model to process articles in foreign languages
- Splitting into cases $\rightarrow$ trial users (more hallucinations for news summarisation) & professional users (more accurate and reliable for news summarisation)
- Training model further to more efficiently
- Raising appropriate errors when given invalid inputs



In [ ]:
%%capture
# install all libraries
!pip install alpha_vantage
!pip install requests
!pip install newspaper3k
!pip install cohere

In [ ]:
%%capture
#imports
import nltk
import requests
nltk.download('punkt')
from newspaper import Article
import random
import cohere
from newspaper.images import urllib
import json

In [14]:
def get_yahoo_shortname(symbol): #function to lookup company ticker's name
    response = urllib.request.urlopen(f'https://query2.finance.yahoo.com/v1/finance/search?q={symbol}')
    content = response.read()
    company = json.loads(content.decode('utf8'))['quotes'][0]['shortname']
    return company

def sentimental_analysis(): #big boy function
    """
    Returns sentimental analysis on company ticker, using Cohere and Alpha Vantage.
    """

    cohere_apikey = str(input("What is your Cohere API key?\n"))
    alpha_vantage = str(input("What is your AlphaVantage API key?\n"))
    ticker = str(input("What is your prospective company's ticker symbol?\n"))
    limit = str(input("How many articles do you want to analyse? (Up to 200 for trial users)\n"))

    co = cohere.Client(cohere_apikey)

    url = ('https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=' 
           + f"{ticker}"+ f'&limit={limit}'+ '&apikey=' + f"{alpha_vantage}")
    r = requests.get(url)
    data = r.json()

    article_urls = [] #initialising article url list and total ticker sentiment score
    total_ticker_sentiment = 0.0

    for a in range(0, len(data['feed'])): #Parses through each article
        article_urls.append(data['feed'][a]['url']) #Appends url to the list
        for t in range(0, len(data['feed'][a]['ticker_sentiment'])): #Parses through ticker list for particular ticker
            if data['feed'][a]['ticker_sentiment'][t]['ticker'] == ticker: #Only grab the info about relevant ticker
              total_ticker_sentiment += float(data['feed'][a]['ticker_sentiment'][t]['ticker_sentiment_score'])

    average_ticker_sentiment = total_ticker_sentiment/len(data['feed'])

    if average_ticker_sentiment <= -0.35:
      sentiment = "Bearish"
    elif average_ticker_sentiment <= -0.15:
      sentiment = "Slightly Bearish"
    elif average_ticker_sentiment < 0.15:
      sentiment = "Neutral"
    elif average_ticker_sentiment < 0.35:
      sentiment = "Slightly Bullish"
    else:
      sentiment = "Bullish" 

    article_contents = []
    company = get_yahoo_shortname(ticker)

    for url in article_urls:
      try: #Web scraping is not always legal and allowed, so we will do it for websites that allow it.
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        article_contents.append(article.text)
      except:
        continue #we skip summarizing the articles that do not allow for scraping
      
# Cutting all this out since I only have trial API key. (Use if premium plan)

#    random_list_first = random.sample(range(0,len(article_contents)), 
#                                      k=(len(article_contents)//5)) #RNG for summarizing
   
#    summarized_articles = []

#    for i in random_list_first: 
#      response = co.summarize(text=article_contents[i], model='summarize-xlarge', 
#                                length='long', format="paragraph", temperature=0.1,
#                                additional_command=f"Summarise with the focus on the company: {company}")
#      summarized_articles.append(response.summary)
    
#    random_list_second = random.sample(range(0,len(summarized_articles)),
#                                       k=len((summarized_articles)//2)) #second RNG
#    second_random_articles = []

#    for i in random_list_second: 
#      response = co.summarize(text=summarized_articles[i], model='summarize-xlarge', 
#                                length='long', format="paragraph", temperature=0.1,
#                                additional_command=f"Summarise with the focus on the company: {company}")
#      second_random_articles.append(response.summary)

#    random_list_final = random.sample(range(0,len(second_random_articles)),
#                                      k=1)
    
#    for i in random_list_final:
#        response = co.summarize(text=second_random_articles[i], model='summarize-xlarge', 
#                                length='long', format="paragraph", temperature=0.1,
#                                additional_command=f"Summarise with the focus on the company: {company}")
    
    random_list_first = random.sample(range(0,len(article_contents)), 
                                      k=min(3,len(article_contents))) #RNG for summarizing
   
    summarized_articles = []

    for i in random_list_first: 
      response = co.summarize(text=article_contents[i], model='summarize-xlarge', 
                                length='long', format="paragraph", temperature=0.1,
                                additional_command=f"Write as an equity research analyst that is researching on the current news of {company}")
      summarized_articles.append(response.summary)
    
    big_string = summarized_articles[0]

    for i in range(1,len(big_string)):
      big_string += "\n" #doing this to save on memory
      big_string += big_string[i]

    response = co.summarize(text=big_string, model='summarize-xlarge', 
                            length='long', format="paragraph", temperature=0.1,
                            additional_command=f"Write as an equity research analyst that is researching on the current news of {company}")

    return sentiment, response.summary

statement = sentimental_analysis()

print('\n', 'Overall recent market sentiment: ',
      '\n', statement[0], '\n', 
      'Summarization of current market news: ', '\n',
      statement[1])


        



What is your Cohere API key?
TqNim7S803C7Pz7pLmGYb38PZkiDeceA1wkFDDTf
What is your AlphaVantage API key?
KK6JVIIT1YZ51460
What is your prospective company's ticker symbol?
GOOG
How many articles do you want to analyse? (Up to 200 for trial users)
20

 Overall recent market sentiment:  
 Neutral 
 Summarization of current market news:  
 Alphabet Inc has added a new update to its music app that lets users automatically download recently played songs on Android. The new update allows users to download up to 200 recently played songs. The users may also find this feature already enabled on their devices. Meanwhile, Alphabet has announced that US creators can now create podcasts in YouTube Studio and the inclusion of podcasts in the company's music app is coming soon.
